In [1]:
using Luxor, LightXML

In [2]:
drawing_width = 300
drawing_height = 375;

In [6]:
function textlabel!(parent, txt::AbstractString, pos::Point; xoffset::Number=0, yoffset::Number=0)
    x = pos.x + xoffset
    y = pos.y + yoffset
    elem = new_child(parent, "text")
    add_text(elem, txt)
    set_attributes(elem, Dict("class"=>"math", "x"=>string(x, "px"), "y"=>string(y, "px")))
    return nothing
end;

In [24]:
Drawing(drawing_width, drawing_height, :svg)

# box(O, Point(drawing_width, drawing_height), :stroke)
bbox = BoundingBox(;centered=false)

origin(Point(drawing_width/2, drawing_height-50))

pt_A = O + (50, -150)
v_A = pt_A - (75, 75)
v_A_t = pt_A - (50, 150)
local_horiz = perpendicular(pt_A, O, 175)
baseline = slope(pt_A, local_horiz)

@layer begin
    setcolor("black")
    circle(O, 7, :fill)
    circle(pt_A, 3, :fill)
    arrow(O, pt_A)
    arrow(pt_A, 110, baseline, slope(pt_A, v_A), clockwise=true)
    arrow(pt_A, 165, baseline, slope(pt_A, v_A_t), clockwise=true)
    arrow(pt_A, 75, slope(pt_A, v_A), slope(pt_A, v_A_t), clockwise=true)
    arrow(pt_A, 75, slope(pt_A, v_A_t), slope(pt_A, v_A), clockwise=false)
    origin(getworldposition(last(intersectionlines(pt_A, local_horiz, v_A, v_A_t)), centered=false))
    arrow(O, 60, baseline, slope(v_A, v_A_t), clockwise=true)
end

function ellipse(nu, scale, e)
    return scale / (1 + e*cos(nu))
end

@layer begin
    setcolor("black")
    arrow(pt_A, v_A)
    @layer begin
        setline(1)
        line(pt_A, local_horiz, :stroke)
    end
    poly(polar.(ellipse.(deg2rad.(45:1:95), 164, 0.1), deg2rad.(-45:-1:-95)), :stroke)
    @layer begin
        setline(1)
        origin(getworldposition(pt_A, centered=false))
        pt_1 = polar(15, baseline)
        pt_2 = polar(sqrt(2)*15, baseline-pi/4)
        pt_3 = polar(15, baseline-pi/2)
        line(pt_1, pt_2, :stroke)
        line(pt_2, pt_3, :stroke)
    end
end

@layer begin
    setcolor(141/255, 160/255, 203/255, 1.0)
    arrow(pt_A, v_A_t)
    poly(polar.(ellipse.(deg2rad.(45:1:95), 188, 0.6), deg2rad.(-45:-1:-95)), :stroke)
    text("Transfer Orbit", pt_A + (30, 60), valign=:top, halign=:center)
end

@layer begin
    setcolor(102/255,194/255,165/255,1.0)
    arrow(v_A, v_A_t)
end

@layer begin
    setdash("dashed")
    line(v_A_t, v_A_t - (6, tan(slope(pt_A, v_A_t))*6), :stroke)
    line(v_A, v_A - (10, tan(slope(pt_A, v_A))*10), :stroke)
end

text("Local Horizon", local_horiz + (0, 4), angle=pi+baseline, valign=:top)
label("Initial Orbit", :E, pt_A, offset=17)

finish()

svg_out = svgstring();
xdoc = parse_string(svg_out)
xroot = root(xdoc);
textlabel!(xroot, "\\boldsymbol{r}_A", between(O, pt_A, 0.4), xoffset=130, yoffset=300)
textlabel!(xroot, "A", pt_A, xoffset=150, yoffset=300)
textlabel!(xroot, "\\phi_A", pt_A, xoffset=30, yoffset=260)
textlabel!(xroot, "\\phi_{A_t}", pt_A, xoffset=0, yoffset=200)
textlabel!(xroot, "\\Delta\\phi", pt_A, xoffset=95, yoffset=240)
textlabel!(xroot, "\\gamma", pt_A, xoffset=50, yoffset=220)
textlabel!(xroot, "\\Delta\\boldsymbol{v}", pt_A, xoffset=65, yoffset=200)
textlabel!(xroot, "\\boldsymbol{v}_{A_t}", pt_A, xoffset=110, yoffset=190)
textlabel!(xroot, "\\boldsymbol{v}_{A}", pt_A, xoffset=64, yoffset=260)
save_file(xdoc, "../raw_svg/non-hohmann-delta-v.svg");